In [9]:
import pandas as pd
from pathlib import Path
base_output = Path('group_results')

group_res = {}
group_f = {}

for fp in base_output.rglob('group_stats.csv'):
    group_res[fp.parent.stem] = pd.read_csv(
        fp,
        index_col=0,
        header=[0])
    group_f[fp.parent.stem] = pd.read_csv(
        fp.parent / 'group_fstats.csv',
        index_col=0,
        header=[0])
    
combined_res = {key: pd.concat((group_res[key].loc[:, ['COPE', 'VARCOPE', 'z']] , group_f[key]), axis=1) for key in group_res}


def print_percentage_activations(df):
    mean_activation_df = df.filter(regex='mean_activation', axis=0)
    mean_activation_df.index = mean_activation_df.index.str.replace('_mean_activation','').str.replace('conc_','')
    mean_activation_df = mean_activation_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
    mean_activation_df

    constant_df = df.filter(regex='constant', axis=0)
    constant_df.index = constant_df.index.str.replace('_constant','').str.replace('conc_','')
    constant_df = constant_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
    constant_df.COPE

    mean_activation_df
    # constant_df
    mean_activation_df.loc[:, ['COPE', 'VARCOPE']] = mean_activation_df.loc[:, ['COPE', 'VARCOPE']].div(constant_df.COPE, axis=0)

    return mean_activation_df.style\
    .format(formatter={
        'COPE': "{:+0.1%}",
        'VARCOPE': "{:+0.2%}",
        'z': "{:+05.2f}",
        'p': "{:05.3f}"})\
    .highlight_between(subset=['p'],left=0, right=0.05, props='font-weight:bold;color:#e83e8c')


In [10]:
combined_res

{'3T_fixed_lb':                                    COPE       VARCOPE          z       f-stat  \
 conc_Ala_STIM0                -0.000495  2.474674e-08  -3.083774     9.914938   
 conc_Ala_STIM1                -0.000417  2.473649e-08  -2.609997     7.026319   
 conc_Ala_constant              0.000220  6.212306e-09   2.741949     7.776592   
 conc_Asc_STIM0                 0.000476  1.585917e-08   3.675036    14.308921   
 conc_Asc_STIM1                 0.000456  1.581167e-08   3.530734    13.152238   
 ...                                 ...           ...        ...          ...   
 conc_sIns_mean_activation     -0.000016  3.792249e-10  -0.834378     0.700859   
 conc_NAA+NAAG_mean_activation  0.004865  2.401869e-09  23.510939  9853.004883   
 conc_Cr+PCr_mean_activation    0.003104  1.334232e-09  22.670650  7221.461426   
 conc_PCh+GPC_mean_activation   0.000395  2.402855e-10  15.131670   648.217102   
 conc_Glu+Gln_mean_activation   0.003272  9.379709e-09  17.067413  1141.053101   
 

In [11]:
print(group_f.keys())

dict_keys(['3T_fixed_lb', '7T_fixed_lb', '3T_exact_lb', '7T_exact_lb'])


In [14]:
def print_percentage_activations_all(all_df):
    out = {}
    for key, df in all_df.items():
        mean_activation_df = df.filter(regex='mean_activation', axis=0)
        mean_activation_df.index = mean_activation_df.index.str.replace('_mean_activation','').str.replace('conc_','')
        mean_activation_df = mean_activation_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
        mean_activation_df

        constant_df = df.filter(regex='constant', axis=0)
        constant_df.index = constant_df.index.str.replace('_constant','').str.replace('conc_','')
        constant_df = constant_df.drop(['NAA', 'NAAG', 'Cr', 'PCr', 'PCh', 'GPC'])
        constant_df.COPE

        mean_activation_df
        # constant_df
        mean_activation_df.loc[:, ['COPE', 'VARCOPE']] = mean_activation_df.loc[:, ['COPE', 'VARCOPE']].div(constant_df.COPE, axis=0)
        out[key] = mean_activation_df.loc[:, ['COPE', 'z', 'f-stat', 'zf-stat','p']]
    
    out = pd.concat(out.values(), axis=1, keys=out.keys())

    formatter_dict = {}
    p_list = []
    for key in all_df:
        formatter_dict.update({
            (key, 'COPE'): "{:+0.1%}",
            #(key, 'VARCOPE'): "{:+0.2%}",
            (key, 'z'): "{:+0.2f}",
            (key, 'f-stat'): "{:+0.2f}",
            (key, 'zf-stat'): "{:+0.2f}",
            (key, 'p'): "{:0.1e}"})
        p_list.append((key, 'p'))
    return out.style\
    .format(formatter=formatter_dict)\
    .highlight_between(subset=p_list,left=0, right=0.05, props='font-weight:bold;color:#e83e8c')

print_percentage_activations_all(combined_res)

In [27]:
print_percentage_activations(group_res['3T_fixed_lb'])

,COPE,VARCOPE,z,p
Ala,-500.7%,+0.00%,-15.41,1.000
Asc,+2.6%,+0.00%,+9.31,0.000
Asp,-2.3%,+0.00%,-14.14,1.000
GABA,-21.0%,+0.00%,-26.29,1.000
GSH,+1.5%,+0.00%,+11.83,0.000
Glc,-14.8%,+0.00%,-20.09,1.000
Gln,-3.5%,+0.00%,-19.34,1.000
Glu,+3.5%,+0.00%,+23.82,0.000
Gly,+402.3%,+0.00%,+17.16,0.000
Ins,+1.0%,+0.00%,+22.43,0.000


In [8]:
print_percentage_activations(group_res['3T_exact_lb'])

,COPE,VARCOPE,z,p
Ala,-207.8%,+0.01%,-2.83,0.998
Asc,+1.1%,+0.00%,+1.82,0.034
Asp,+0.1%,+0.00%,+0.43,0.333
GABA,-6.4%,+0.00%,-8.08,1.000
GSH,+0.4%,+0.00%,+1.21,0.112
Glc,-5.5%,+0.00%,-6.19,1.000
Gln,-1.3%,+0.00%,-5.31,1.000
Glu,+1.1%,+0.00%,+10.31,0.000
Gly,+20.6%,+0.00%,+1.26,0.104
Ins,+0.3%,+0.00%,+3.64,0.000


In [9]:
print_percentage_activations(group_res['7T_fixed_lb'])

,COPE,VARCOPE,z,p
Ala,-24.3%,+0.00%,-0.54,0.705
Asc,+6.1%,+0.00%,+14.26,0.000
Asp,-0.7%,+0.00%,-4.59,1.000
GABA,-18.3%,+0.00%,-20.49,1.000
GSH,+1.7%,+0.00%,+10.58,0.000
Glc,-23.0%,+0.00%,-20.44,1.000
Gln,-0.8%,+0.00%,-7.30,1.000
Glu,+2.7%,+0.00%,+22.65,0.000
Gly,-inf%,+inf%,-15.76,1.000
Ins,+2.0%,+0.00%,+21.19,0.000


In [10]:
print_percentage_activations(group_res['7T_exact_lb'])

,COPE,VARCOPE,z,p
Ala,-32.2%,+0.00%,-1.61,0.946
Asc,+0.8%,+0.00%,+2.86,0.002
Asp,+0.4%,+0.00%,+2.79,0.003
GABA,-3.7%,+0.00%,-9.41,1.000
GSH,+0.5%,+0.00%,+3.60,0.000
Glc,-5.4%,+0.00%,-10.04,1.000
Gln,-0.3%,+0.00%,-2.88,0.998
Glu,+0.7%,+0.00%,+14.02,0.000
Gly,-1539.5%,+0.00%,-14.08,1.000
Ins,+0.7%,+0.00%,+14.61,0.000


In [12]:
group_res['7T_exact_lb'].filter(regex='gamma', axis=0)

,COPE,VARCOPE,z,p
gamma_0_constant,0.321668,0.000029,20.634192,6.769526e-95
gamma_0_Lorentz_lb,2.407722,0.000361,24.779160,7.520519e-136


In [13]:
group_res['3T_exact_lb'].filter(regex='gamma', axis=0)

,COPE,VARCOPE,z,p
gamma_0_constant,0.651362,0.000038,23.875864,2.728432e-126
gamma_0_Lorentz_lb,2.191931,0.001509,20.319874,4.289831e-92
